In [ ]:
        # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader

2024-07-02 14:27:20.096760: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 14:27:20.096867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 14:27:20.232650: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
dataset = load_dataset('imdb')
print(dataset)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [3]:
from transformers import BertTokenizer

- the tokenizer function which is pretrained in a specific format (Adding CLS and SEP and PAD tokens during the sentences for correct undertsanding by the model)

 - The padding is added to make sure that each and every sentense is of the same length (PAD tokens are added if the sentense is short) longer sentences are cutt off
 
 - Tokenizer is then make to map on each and every sentense in the dataset with batch processing for performance.

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenizer_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenizer_dataset = dataset.map(tokenizer_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
train_valid = tokenizer_dataset['train'].train_test_split(test_size = 0.2)
train_dataset = train_valid['train']
valid_dataset = train_valid['test']

In [6]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size  =8)
valid_dataloader = DataLoader(valid_dataset,batch_size=8)

In [7]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
training_args = TrainingArguments(
    output_dir = './results/',
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 3,
    weight_decay = 0.01,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = valid_dataset
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.256700,0.356660
2,0.138800,0.279320
3,0.061200,0.357664


TrainOutput(global_step=7500, training_loss=0.1731505803426107, metrics={'train_runtime': 3736.3361, 'train_samples_per_second': 16.059, 'train_steps_per_second': 2.007, 'total_flos': 1.57866633216e+16, 'train_loss': 0.1731505803426107, 'epoch': 3.0})

In [13]:
metrics = trainer.evaluate()

In [14]:
print(metrics)

{'eval_loss': 0.35766366124153137, 'eval_runtime': 84.3751, 'eval_samples_per_second': 59.259, 'eval_steps_per_second': 7.407, 'epoch': 3.0}


In [15]:
predictions = trainer.predict(valid_dataset)

In [17]:
from sklearn.metrics import accuracy_score

preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids
test_accuracy = accuracy_score(labels, preds)
print(f"Test accuracy : {test_accuracy}")

Test accuracy : 0.9328
